In [6]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

#### surprise 데이터 로딩
* Dataset클래스를 이용해서만 가능
    -  Row레벨형태로 된 포맷의 데이터만 처리

In [7]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

In [8]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

#### row레벨의 사용자-아이템 평점데이터 전치행렬하였지만  surprise에서는 자체적 변경함
* 그대로 사용자-아이템 평점데이터 데이터셋 적용

In [9]:
algo = SVD()
algo.fit(trainset) 

In [10]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4799408022291747, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.60958495178257, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.013294495076437, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6542794819446733, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.460750203911854, details={'was_impossible': False})]

In [12]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.4799408022291747, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.60958495178257, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.013294495076437, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6542794819446733, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.460750203911854, details={'was_impossible': False})]

In [13]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.4799408022291747),
 ('882', '291', 3.60958495178257),
 ('535', '507', 4.013294495076437)]

In [ ]:
### surprise 패키지의 다른 추천예측 메서드인 predict() 이용
* 개별사용자의 아이템에 대한 추천 평점을 예측해 줌
* test()메서드는 predict()반복적으로 수행한 결과

In [14]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.33   {'was_impossible': False}


In [15]:
# 테스트 데이터 셋으로 추천예측평점과 실제평점과의 차이를 평가
# surprise의 accuracy모듈은 RMSE, MSE 등의 방법으로 추천시스템의 성능평가 정보 제공
accuracy.rmse(predictions)

RMSE: 0.9496


0.949638985662199

In [ ]:
#### Surprise 모듈 소개

* Dataset.load_builtin,
* Dataset.load_from_file(file_path, reader) : OS파일에서 데이터 로딩시, 콤마 탭 등으로 칼럼 분리된 포맷의 OS파일
* Dataset.load_from_df(df, reader) : 판다스로 로딩, 
* 데이터셋 칼럼순서가 사용자아이디, 아이템아이디, 평점순으로 반드시
* 헤더문자열 있으면 제거해야 함
* 무비렌즈 데이터형식을 따라야 함
    - 아닌경우 Reader클래스 설정해 주어야 함 
        + line_format(string): 칼럼을 순서대로 나열, 입력된 문자열을 공백으로 칼럼으로 인식
        + sep(char) : 칼럼을 분리하는 분리자 디폴트 \t, 판다스 df에서 입력받을 경우 기재할 필요없음
        + rating_scale(tuple, optional) : 평점값의 최소 ~ 최대평점 설정, 디폴트 1.5, 


In [16]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)



Bad key "axes_unicode_minus" on line 3 in
/home/jovyan/.config/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [17]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./ml-latest-small/ratings_noh.csv',reader=reader)

In [18]:
# SVD 행렬분해 기법
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

In [ ]:
#### 판다스로 로딩

In [19]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

In [ ]:
### Surprise 추천 알고리즘 클래스
* 예측성능 벤치마크 결과 : http://surpriselib.com
* SVD : 행렬분해를 통한 잠재요인 협업필터링을 위한 SVD 알고리즘
    - n_factors : 잠재요인 K의 개수, 디폴트 100, 커질수록 정확도 높으나 과적합 가능성
    - n_epochs : SGD 수행시 반복횟수, 디폴트 20
    - biased : 사용자편향사용여부  디폴트 True(그냥 유지)
* KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN알고리즘
* BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘
* SVD++, NMF,,,,
* http://surprise.readthedocs.io/en/stable/prediction_algrithms_package.html

In [ ]:
# Baseline 평점
* 개인의성향(평가에 대한 인간적인 요소 등)을 편향성(Bias)요소로 반영해 평점을 부과
* 전체평균평점 + 사용자편향점수 + 아이템 편향 점수
    - 전제평균평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자편향점수 : 사용자별 아이템 평점 평균값 - 전체평균평점
    - 아이템편향점수 : 아이템별평점평균값 - 전체평균평점

### 교차 검증(Cross Validation)과 하이퍼 파라미터 튜닝 

In [21]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('./ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 

# 교차검증
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8767  0.8731  0.8763  0.8703  0.8707  0.8734  0.0027  
MAE (testset)     0.6738  0.6716  0.6732  0.6659  0.6715  0.6712  0.0028  
Fit time          5.60    5.53    5.55    5.52    5.55    5.55    0.03    
Test time         0.14    0.18    0.20    0.13    0.19    0.17    0.03    


{'test_rmse': array([0.87667764, 0.87310527, 0.87631204, 0.87033813, 0.87066121]),
 'test_mae': array([0.67377618, 0.67164886, 0.67316122, 0.66593288, 0.67154189]),
 'fit_time': (5.59583854675293,
  5.533417463302612,
  5.5481884479522705,
  5.515202522277832,
  5.548883676528931),
 'test_time': (0.13613080978393555,
  0.18450379371643066,
  0.2043306827545166,
  0.1336348056793213,
  0.1929025650024414)}

In [22]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])




0.8769220548378276
{'n_epochs': 20, 'n_factors': 50}


### Surprise 를 이용한 개인화 영화 추천 시스템 구축 ===>  평점을 매기지 않은 영화중 개인취향에 적절한 영화 추천, 평점 예측
* 지금까지 surprise는 학습데이터로 fit()호출해 학습한 뒤 
* 테스트데이터로 test() 호출해 예측 평점 -> MSE/RMSE 성능평가
* 이제는 surprise패키지로 학습된 알고리즘으로 특정사용자가 평점을 매기지 않은 영화중 개인취향에 적절한 영화 추천


In [23]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다. => DatasetAutoFolds클래스 이용
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)


AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [24]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


In [25]:
# SVD
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


In [ ]:
# 사용자 9, 영화아이디 42에 대해

In [26]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [27]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)


user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


## 사용자가 평점을 매기지 않은 전체영화를 추출한 뒤 예측평점순으로 영화추천

In [28]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [32]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict()메서드를 평점이 없는 영화에 반복수행한 후 list저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions객체를 원소로 가지고 있음.
    # [Predictions(uid='9', iid='1', est='3.69'),  Predictions( ,,,)]
    # 이를 est값으로 정렬하기 위하여 아래의 sortkey_est함수를 정의함
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est()반환값의 내림차순으로 정렬수행하고 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    # top_n으로 추출된 영화의 정보추출, 영화아이디, 추천예상평점, 제목추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions ]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating) ]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
